In [83]:
import numpy as np # handle data in a vectorized manner

import json # library to handle JSON files

import pandas as pd # data analysis

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [84]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [85]:
with open('newyork_data.json') as json_data:
    ny_data = json.load(json_data)

In [86]:
neighborhoods_data = ny_data['features']

In [181]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [88]:
!wget -q -O 'New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv' https://data.cityofnewyork.us/api/views/swpk-hqdp/rows.csv?accessType=DOWNLOAD
print('Data downloaded!')

Data downloaded!


In [89]:
pop_ny = pd.read_csv('New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv')

In [90]:
pop_ny.rename(columns={'NTA Name':'Neighborhood'},inplace=True)

In [91]:
pop_ny1 = pop_ny.loc[(pop_ny['Year']) == 2010]

In [182]:
pop_ny2 = pop_ny1.loc[:,['Borough', 'Neighborhood', 'Population']]
pop_ny2.head()

,Borough,Neighborhood,Population
195,Bronx,Claremont-Bathgate,31078
196,Bronx,Eastchester-Edenwald-Baychester,34517
197,Bronx,Bedford Park-Fordham North,54415
198,Bronx,Belmont,27378
199,Bronx,Bronxdale,35538


In [93]:
df_merge = pd.merge(pop_ny2, neighborhoods, on='Neighborhood')
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Borough_x':'Borough'},inplace=True)
df_merge.drop(['Borough_y'],axis=1,inplace=True)
df_merge.head()

,Borough,Neighborhood,Population,Latitude,Longitude
0,Bronx,Belmont,27378,40.857277,-73.888452
1,Bronx,Bronxdale,35538,40.852723,-73.861726
2,Bronx,East Tremont,43423,40.842696,-73.887356
3,Bronx,Hunts Point,27204,40.809730,-73.883315
4,Bronx,Kingsbridge Heights,32496,40.870392,-73.901523


In [94]:
address = 'New York, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York are 40.7127281, -74.0060152.


In [95]:
CLIENT_ID = 'VBQJLEG4YQ1S1MTUQKEKRGRMUBUT3MZPGMJQZI1TCIKBBYCK' # your Foursquare ID
CLIENT_SECRET = 'J22NGQIVVBQ2AJ4IN14HOHQLJ5QX040YXIDZ03LOWT3KCYSB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VBQJLEG4YQ1S1MTUQKEKRGRMUBUT3MZPGMJQZI1TCIKBBYCK
CLIENT_SECRET:J22NGQIVVBQ2AJ4IN14HOHQLJ5QX040YXIDZ03LOWT3KCYSB


In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
ny_venues = getNearbyVenues(names=df_merge['Neighborhood'],
                                   latitudes=df_merge['Latitude'],
                                   longitudes=df_merge['Longitude'],
                                  )

Belmont


KeyError: 'groups'

In [52]:
ny_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Belmont,40.857277,-73.888452,Tino's Delicatessen,40.855882,-73.887166,Italian Restaurant
1,Belmont,40.857277,-73.888452,Casa Della Mozzarella,40.855440,-73.887373,Deli / Bodega
2,Belmont,40.857277,-73.888452,Full Moon Pizzeria,40.855506,-73.887557,Pizza Place
3,Belmont,40.857277,-73.888452,Zero Otto Nove,40.854714,-73.888388,Pizza Place
4,Belmont,40.857277,-73.888452,Michaelangelo's Coal Fired Brick Oven Pizza Re...,40.857412,-73.886468,Italian Restaurant
5,Belmont,40.857277,-73.888452,DeLillo Pastry Shop,40.855364,-73.887198,Dessert Shop
6,Belmont,40.857277,-73.888452,Gino's Pastry Shop,40.855648,-73.888196,Dessert Shop
7,Belmont,40.857277,-73.888452,Cosenza's Fish Market,40.854660,-73.888335,Fish Market
8,Belmont,40.857277,-73.888452,Teitel Brothers Imported Italian Specialty Pro...,40.854744,-73.888220,Food & Drink Shop
9,Belmont,40.857277,-73.888452,Madonia Bakery,40.854412,-73.888516,Bakery


In [122]:
ny_venues
ny_venues1 = ny_venues.loc[(ny_venues['Venue Category'] == 'Japanese Restaurant')
                           |(ny_venues['Venue Category'] == 'Chinese Restaurant')
                          |(ny_venues['Venue Category'] == 'Korean Restaurant')
                           |(ny_venues['Venue Category'] == 'Cambodian Restaurant')
                           |(ny_venues['Venue Category'] == 'Filipino Restaurant')
                           |(ny_venues['Venue Category'] == 'Himalayan Restaurant')
                            |(ny_venues['Venue Category'] == 'Indonesian Restaurant')
                            |(ny_venues['Venue Category'] == 'Malay Restaurant')
                            |(ny_venues['Venue Category'] == 'Tibetan Restaurant')
                            |(ny_venues['Venue Category'] == 'Vietnamese Restaurant')
                            |(ny_venues['Venue Category'] == 'Thai Restaurant')
                            |(ny_venues['Venue Category'] == 'Satay Restaurant')
                            |(ny_venues['Venue Category'] == 'Mongolian Restaurant')    
                           
                          ]

In [140]:
ny_venues1.rename(columns={'Venue Category':'Number of Asians Restaurants'},inplace=True)
ny_venues1.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude', 'Venue Longitude'],axis=1,inplace=True)

,Number of Asians Restaurants
Neighborhood,
Astoria,5
Auburndale,1
Bath Beach,3
Bay Ridge,6
Bellerose,2
Belmont,2
Brighton Beach,1
Bronxdale,1
Brownsville,2


In [144]:
ny_venues2 = ny_venues1.groupby('Neighborhood').count()

In [167]:
df_merge2 = pd.merge(df_merge, ny_venues2, on='Neighborhood')
df_merge2

,Borough,Neighborhood,Population,Latitude,Longitude,Number of Asians Restaurants
0,Bronx,Belmont,27378,40.857277,-73.888452,2
1,Bronx,Bronxdale,35538,40.852723,-73.861726,1
2,Bronx,Kingsbridge Heights,32496,40.870392,-73.901523,2
3,Bronx,Longwood,26196,40.815099,-73.895788,1
4,Bronx,Norwood,40494,40.877224,-73.879391,1
5,Bronx,Parkchester,29821,40.837938,-73.856003,1
6,Bronx,Pelham Parkway,30073,40.857413,-73.854756,2
7,Brooklyn,Brighton Beach,35547,40.576825,-73.965094,1
8,Brooklyn,West Brighton,17750,40.631879,-74.107182,2
9,Brooklyn,Homecrest,44316,40.598525,-73.959185,5


In [168]:
#Foursquare has a limit on number of venues, we need to scale up these numbers
df_merge2['Number of Asians Restaurants'] = round(df_merge2['Number of Asians Restaurants']*11.1)
df_merge2

,Borough,Neighborhood,Population,Latitude,Longitude,Number of Asians Restaurants
0,Bronx,Belmont,27378,40.857277,-73.888452,22.0
1,Bronx,Bronxdale,35538,40.852723,-73.861726,11.0
2,Bronx,Kingsbridge Heights,32496,40.870392,-73.901523,22.0
3,Bronx,Longwood,26196,40.815099,-73.895788,11.0
4,Bronx,Norwood,40494,40.877224,-73.879391,11.0
5,Bronx,Parkchester,29821,40.837938,-73.856003,11.0
6,Bronx,Pelham Parkway,30073,40.857413,-73.854756,22.0
7,Brooklyn,Brighton Beach,35547,40.576825,-73.965094,11.0
8,Brooklyn,West Brighton,17750,40.631879,-74.107182,22.0
9,Brooklyn,Homecrest,44316,40.598525,-73.959185,56.0


In [172]:
df_merge2['Restaurants/Population'] = df_merge2['Number of Asians Restaurants']/df_merge2['Population']

In [173]:
df_merge2

,Borough,Neighborhood,Population,Latitude,Longitude,Number of Asians Restaurants,Population/Restaurants,Restaurants/Population
0,Bronx,Belmont,27378,40.857277,-73.888452,22.0,1244.454545,0.000804
1,Bronx,Bronxdale,35538,40.852723,-73.861726,11.0,3230.727273,0.000310
2,Bronx,Kingsbridge Heights,32496,40.870392,-73.901523,22.0,1477.090909,0.000677
3,Bronx,Longwood,26196,40.815099,-73.895788,11.0,2381.454545,0.000420
4,Bronx,Norwood,40494,40.877224,-73.879391,11.0,3681.272727,0.000272
5,Bronx,Parkchester,29821,40.837938,-73.856003,11.0,2711.000000,0.000369
6,Bronx,Pelham Parkway,30073,40.857413,-73.854756,22.0,1366.954545,0.000732
7,Brooklyn,Brighton Beach,35547,40.576825,-73.965094,11.0,3231.545455,0.000309
8,Brooklyn,West Brighton,17750,40.631879,-74.107182,22.0,806.818182,0.001239
9,Brooklyn,Homecrest,44316,40.598525,-73.959185,56.0,791.357143,0.001264


In [175]:
df_merge2 = df_merge2.loc[(df_merge2['Borough'] == 'Manhattan')|(df_merge2['Borough'] == 'Brooklyn')]

In [176]:
df_merge2

,Borough,Neighborhood,Population,Latitude,Longitude,Number of Asians Restaurants,Population/Restaurants,Restaurants/Population
7,Brooklyn,Brighton Beach,35547,40.576825,-73.965094,11.0,3231.545455,0.000309
8,Brooklyn,West Brighton,17750,40.631879,-74.107182,22.0,806.818182,0.001239
9,Brooklyn,Homecrest,44316,40.598525,-73.959185,56.0,791.357143,0.001264
10,Brooklyn,Gravesend,29436,40.595260,-73.973471,44.0,669.000000,0.001495
11,Brooklyn,Bath Beach,29931,40.599519,-73.998752,33.0,907.000000,0.001103
12,Brooklyn,Bay Ridge,79371,40.625801,-74.030621,67.0,1184.641791,0.000844
13,Brooklyn,Windsor Terrace,20988,40.656946,-73.980073,11.0,1908.000000,0.000524
14,Brooklyn,Flatbush,105804,40.636326,-73.958401,11.0,9618.545455,0.000104
15,Brooklyn,Canarsie,83693,40.635564,-73.902093,11.0,7608.454545,0.000131
16,Brooklyn,Prospect Heights,19849,40.676822,-73.964859,67.0,296.253731,0.003375


In [178]:
df_merge2.sort_values(by='Restaurants/Population')

,Borough,Neighborhood,Population,Latitude,Longitude,Number of Asians Restaurants,Population/Restaurants,Restaurants/Population
14,Brooklyn,Flatbush,105804,40.636326,-73.958401,11.0,9618.545455,0.000104
23,Brooklyn,East New York,91958,40.669926,-73.880699,11.0,8359.818182,0.000120
15,Brooklyn,Canarsie,83693,40.635564,-73.902093,11.0,7608.454545,0.000131
7,Brooklyn,Brighton Beach,35547,40.576825,-73.965094,11.0,3231.545455,0.000309
24,Brooklyn,East Williamsburg,34158,40.708492,-73.938858,11.0,3105.272727,0.000322
21,Brooklyn,Ocean Hill,31935,40.678403,-73.913068,11.0,2903.181818,0.000344
22,Brooklyn,Brownsville,58300,40.663950,-73.910235,22.0,2650.000000,0.000377
27,Manhattan,Upper West Side,132378,40.787658,-73.977059,56.0,2363.892857,0.000423
13,Brooklyn,Windsor Terrace,20988,40.656946,-73.980073,11.0,1908.000000,0.000524
20,Brooklyn,Greenpoint,34719,40.730201,-73.954241,22.0,1578.136364,0.000634


In [180]:
print ('The best neighborhood to open an asian restaurant is Flatbush, but there are other options such as Canarsie and Upper West Side.')

The best neighborhood to open an asian restaurant is Flatbush, but there are other options such as Canarsie and Upper West Side.
